In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss 

In [ ]:
df = pd.read_csv('/kaggle/input/quora-question-pairs/train.csv.zip', index_col='id')
df.dtypes

In [ ]:
def countletters(s): 
    if isinstance(s, str): 
        return len(s) 
    else: 
        return -1
    
def wordcount(s):
    if isinstance(s, str): 
        words = s.split(' ')
        return len(words) 
    else: 
        return -1

def searchword(word, s):
    if isinstance(s, str):
        if isinstance(word, str) and word.find(' ') == -1:
            found = s.find(word)
            return found 
        else: 
            return -1
    else: 
        return -1

    
def searchcommonwords(s1, s2):
    found = [];
    words = str(s1).split(' ')
    for w in words:
        if str(s2).find(w) >= 0:
            found.append(w)
    words = str(s2).split(' ')
    for w in words:
        if str(s1).find(w) >= 0:
            if not w in found:
                found.append(w)
    return found

def matchwords(s1, s2): 
    set(s1.split(' ')).intersection(set(s2.split(' '))) 
    
# searchexclusive words
def searchexclusivewords(s1, s2):
    found_s1 = [];
    found_s2 = [];
    words = str(s1).split(' ')
    for w in words:
        if not w in found_s1:
            if str(s2).find(w) == -1:
                found_s1.append(w)
    words = str(s2).split(' ')
    for w in words:
        if not w in found_s2:
            if str(s1).find(w) == -1:
                found_s2.append(w)
    return { "one": found_s1, "two": found_s2 }

In [ ]:
def myparse(df):
    #df['nb_car_q1'] = df['question1'].apply(countletters)
    #df['nb_car_q2'] = df['question2'].apply(countletters)
    df['nb_word_q1'] = df['question1'].apply(wordcount)
    df['nb_word_q2'] = df['question2'].apply(wordcount)
    df['common_words'] = df.apply(lambda x: searchcommonwords(x['question1'], x['question2']), axis=1)
    df['nb_common_words'] = df.apply(lambda x: len(x['common_words']), axis=1)
    df['nb_common_words_perc'] = df.apply(lambda x: int(x['nb_common_words']) / max (int(x['nb_word_q1']), int(x['nb_word_q2'])), axis=1)
    df['exclusive_words'] = df.apply(lambda x: searchexclusivewords(x['question1'], x['question2']), axis=1)
    df['nb_exclusive_words'] = df.apply(lambda x: np.abs(len(x['exclusive_words']['one']) - len(x['exclusive_words']['two'])), axis=1)
    return df #[['nb_common_words_perc', 'nb_exclusive_words', 'is_duplicate']]

In [ ]:
df.dropna(inplace=True)

In [ ]:
df = myparse(df)


In [ ]:
X = df[['nb_common_words_perc', 'nb_exclusive_words']]
Y = df['is_duplicate']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
df.groupby('is_duplicate')['nb_common_words_perc'].mean()

In [ ]:
df.groupby('is_duplicate')['nb_exclusive_words'].mean()

In [ ]:
lr = LogisticRegression() 
lr.fit(X_train, y_train) 
# y_pred = lr.predict(X_test) 
#(comparer y_pred et y_test) 

In [ ]:
lr.coef_, lr.intercept_

In [ ]:
for col, coef in zip(X.columns, lr.coef_[0]): 
    print(col, coef)

In [ ]:
y_pred = lr.predict_proba(X_test)[:,1]

In [ ]:
y_pred

In [ ]:
lr.predict_proba(X_test)

In [ ]:
((y_pred - y_test) ** 2).sum() / X_test.shape[0]

In [ ]:
log_loss(y_pred, y_test)

In [ ]:
df_test = pd.read_csv('/kaggle/input/quora-question-pairs/test.csv')


In [ ]:
df_test.dropna(inplace=True)

In [ ]:
df_test.head(10)

In [ ]:
X_test = myparse(df_test)

In [ ]:
y_pred = lr.predict(X_test[['nb_common_words_perc', 'nb_exclusive_words']])

In [ ]:
y_pred